# Practicum 1
- Basic parsing and iteration of Structure objects in BioPython (PDB module)
- Save selected residues into a new PDB file

### Biopython cookbook
https://biopython.org/DIST/docs/tutorial/Tutorial.html

### Bio.PDB module FAQ
https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ


In [4]:
from Bio.PDB import PDBList, is_aa, PDBIO
from Bio.PDB.PDBParser import PDBParser
from Bio.SeqUtils import IUPACData
from Bio.PDB.PDBIO import Select
from Bio.SeqIO.PdbIO import PdbSeqresIterator

In [5]:
path = ".."
path_data = '../data'
path_figures = '../figures'


import os
if not os.path.exists(path_data) :
  os.mkdir(path_data)
if not os.path.exists(path_figures) :
  os.mkdir(path_figures)

# Parsing

In [6]:
# Input
pdb_id = '1cu4' # Other interesting PDBs 1byi 3k8y 1nww

In [7]:
# Fetch a PDB file from the website to the current dir
# Question: how do you download structures from the PDB?
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdb_id, pdir=path + "/pdb", file_format='pdb')  # Will save a pdbXXXX.ent file

Structure exists: '../pdb/pdb1cu4.ent' 


'../pdb/pdb1cu4.ent'

In [8]:
# Get the SEQRES using SeqIO
# It finds and parse all SEQRES fields inside a PDB file and create SeqRecord objects
with open(path + "/pdb/pdb{}.ent".format(pdb_id)) as f:
  seq_records = (PdbSeqresIterator(f))
  for seq_record in seq_records:
    print(seq_record)    
    break

# Print SeqRecord methods and attributes
print([ele for ele in dir(seq_record) if ele[0] != '_'])

ID: 1CU4:H
Name: 1CU4:H
Description: PDB:1CU4 1CU4
Database cross-references: PDB:1CU4, PDB:1CU4
Number of features: 0
/chain=H
/molecule_type=protein
Seq('KVKLQQSGAELVRSGASVKLSCTASGFNIKDYYIQWVKQRPEQGLEWIGWIDPE...VTS')
['annotations', 'dbxrefs', 'description', 'features', 'format', 'id', 'letter_annotations', 'lower', 'name', 'reverse_complement', 'seq', 'translate', 'upper']


In [9]:
# Create the structure object
pdb_id = '1cu4'
structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "/pdb/pdb{}.ent".format(pdb_id))

# Iterate the structure
for model in structure:
  for chain in model:
    for residue in chain:

      # The "is_aa" function checks whether a residue is an amino acid and not a modified residue, HETATM, ligand, ion, etc.
      if is_aa(residue):

        # "residue.id" is a tuple that contains: i) hetero_flag, ii) position (residue index), iii) insertion_code 
        # "IUPACData" is a package to conver residues names from 3 letters to 1 letter
        print("model {} chain {} residue_id {} resname {} resname_3to1 {}".format(
            model.id, chain.id, residue.id, residue.get_resname(),
            IUPACData.protein_letters_3to1.get(residue.get_resname().capitalize())))
        
        # Print all atoms of a residue
        for atom in residue:
            print("atom {} {} {}".format(atom.id, atom.get_bfactor(), atom.get_coord()))

        break
      break
    break
  break

model 0 chain L residue_id (' ', 1, ' ') resname ASP resname_3to1 D
atom N 24.86 [29.448 15.246 40.477]
atom CA 24.86 [28.298 14.797 41.306]
atom C 24.86 [27.455 16.003 41.672]
atom O 24.86 [27.995 17.067 41.952]
atom CB 34.88 [28.791 14.097 42.565]
atom CG 34.88 [29.503 12.784 42.264]
atom OD1 34.88 [29.417 11.868 43.12 ]
atom OD2 34.88 [30.145 12.664 41.186]


# Cut and save a PDB

For example you may want to extract a fragment corresponding to an important part of the protein (domain).

We use a sentinel mechanism to know when to start and finish parsing since the PDB numbering is not guaranteed to be continuous and residue numbers can contain insertion codes, i.e. multiple residues can have the same numbers and different insertion codes.

In [10]:
# Extract a fragment (domain) excluding hetero and water atoms
# Start end positions of a domain identified in PDB 1CU4
pdb_id = '1cu4'
structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "/pdb/pdb{}.ent".format(pdb_id))

domain_start = (" ", 1, " ")
domain_end = (" ", 106, " ")

domain_residues = []
start_flag = False

# Iterate residues of specific chain (L) and model (0)
for residue in structure[0]['L'].get_residues():  # Model 0, chain L
    if residue.id[0] == " ":  
      # Exclude hetero and water atoms
        # print(residue.id)
        # Get starting position, a piori I don't know where is the first residue
        if residue.id == domain_start:
            start_flag = True

        if start_flag:
            domain_residues.append(residue)
            #print(residue.id)

        # Get ending position
        if residue.id == domain_end:
            break

for residue in domain_residues:
  print(residue)

<Residue ASP het=  resseq=1 icode= >
<Residue VAL het=  resseq=2 icode= >
<Residue VAL het=  resseq=3 icode= >
<Residue MET het=  resseq=4 icode= >
<Residue THR het=  resseq=5 icode= >
<Residue GLN het=  resseq=6 icode= >
<Residue THR het=  resseq=7 icode= >
<Residue PRO het=  resseq=8 icode= >
<Residue LEU het=  resseq=9 icode= >
<Residue SER het=  resseq=10 icode= >
<Residue LEU het=  resseq=11 icode= >
<Residue SER het=  resseq=12 icode= >
<Residue VAL het=  resseq=13 icode= >
<Residue THR het=  resseq=14 icode= >
<Residue ILE het=  resseq=15 icode= >
<Residue GLY het=  resseq=16 icode= >
<Residue GLN het=  resseq=17 icode= >
<Residue PRO het=  resseq=18 icode= >
<Residue ALA het=  resseq=19 icode= >
<Residue SER het=  resseq=20 icode= >
<Residue ILE het=  resseq=21 icode= >
<Residue SER het=  resseq=22 icode= >
<Residue CYS het=  resseq=23 icode= >
<Residue LYS het=  resseq=24 icode= >
<Residue SER het=  resseq=25 icode= >
<Residue SER het=  resseq=26 icode= >
<Residue GLN het=  re

In [11]:
# Save the PDB
class Select(Select):
    def __init__(self, chain_ids=None, residues=None):
        self.chain_ids = chain_ids
        self.residues = residues

    def accept_chain(self, chain):
        return self.chain_ids is None or chain.id in self.chain_ids

    def accept_residue(self, residue):
        return self.residues is None or residue in self.residues

    def accept_atom(self, atom):
        return not atom.is_disordered() or atom.get_altloc() == "A"



In [12]:
# Save a PDB file containing only a list of selected residues
io = PDBIO()
io.set_structure(structure[0])
io.save(path + "/pdb/pdb{}_cut.ent".format(pdb_id), select=Select(residues=domain_residues))


# Exercise

Fetch 1CU4
1. How many chains?
2. Total number of hetero atoms?
3. Total number of water molecules?
4. Which is the index of the last residue of chain H?
5. Total number of observed residues of chain H?
6. Why the total number of residues of  is different from the last index?

Fetch 2KKW
7. Split the PDB into different files, one file per model. (Need to write a new "Select" class to pick specific models)

### Solutions

In [13]:
pdb_id = '1cu4'
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdb_id, pdir=path, file_format='pdb')  # Will save to pdbXXXX.ent
structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "/pdb/pdb{}.ent".format(pdb_id))

In [14]:
#Exercise 1
print("chains", len(structure[0]))

chains 3


In [15]:
#Exercise 2
print("hetero atoms", sum([1 for chain in structure[0] for residue in chain if residue.id[0] != ' ' for atom in residue]))

hetero atoms 9


In [16]:
#Exercise 3
print("water molecules", sum([1 for chain in structure[0] for residue in chain if residue.id[0] == 'W']))
for chain in structure[0]:
  for residue in chain:
    if residue.id[0] != ' ':
      print(residue)

water molecules 9
<Residue HOH het=W resseq=215 icode= >
<Residue HOH het=W resseq=216 icode= >
<Residue HOH het=W resseq=217 icode= >
<Residue HOH het=W resseq=214 icode= >
<Residue HOH het=W resseq=215 icode= >
<Residue HOH het=W resseq=216 icode= >
<Residue HOH het=W resseq=217 icode= >
<Residue HOH het=W resseq=218 icode= >
<Residue HOH het=W resseq=2 icode= >


In [17]:
#Exercise 4
print("last residue of chain H", [residue.id for residue in structure[0]['H'] if residue.id[0] == ' '][-1])


last residue of chain H (' ', 213, ' ')


In [18]:
#Exercise 5
print("total elements in chain H", len(structure[0]['H']))
print("total residues in chain H", len([residue for residue in structure[0]['H'] if is_aa(residue)]))
print("total residues in chain H", len([residue for residue in structure[0]['H'] if residue.id[0] == ' ']))

total elements in chain H 220
total residues in chain H 215
total residues in chain H 215


In [19]:
#Exercise 6
print("some elements are not standard residues", [residue.id for residue in structure[0]['H'] if not is_aa(residue)])
print("some elements are water molecules", [residue.id for residue in structure[0]['H'] if residue.id[0] != ' '])
print("some residues have insertion codes", [residue.id for residue in structure[0]['H'] if residue.id[2] != ' '])

some elements are not standard residues [('W', 214, ' '), ('W', 215, ' '), ('W', 216, ' '), ('W', 217, ' '), ('W', 218, ' ')]
some elements are water molecules [('W', 214, ' '), ('W', 215, ' '), ('W', 216, ' '), ('W', 217, ' '), ('W', 218, ' ')]
some residues have insertion codes [(' ', 52, 'A'), (' ', 82, 'A'), (' ', 82, 'B'), (' ', 82, 'C')]


In [20]:
# Define the model select class
class ModelSelect(Select):
    """
    Select model for BioPython PDB save
    """
    def __init__(self, model_ids):
        self.model_ids = model_ids
        self.chain_ids = None
        self.residues = None

    def accept_model(self, model):
        return (model.serial_num) in self.model_ids

In [21]:
# Fetch 2KKW
pdb_id = '2kkw'
pdbl = PDBList()
pdbl.retrieve_pdb_file(pdb_id, pdir=path + "/pdb", file_format='pdb')  # Will save to pdbXXXX.ent
structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "/pdb/pdb{}.ent".format(pdb_id))

# Save models
io = PDBIO()
io.set_structure(structure)
for model_index, model in enumerate(structure):
    io.save(path + "/pdb/pdb{}_{}.ent".format(pdb_id, model_index + 1), select=ModelSelect([model_index + 1]))